Парсер сайт-аукциона Мешок (meshok.net) для стат. исследований.

Выгружает только данные, нужные для исследований, без телефонов и прочих контактов.

Часть 1. Собирает ссылки и информацию по страницам выдачи, без захода на страницу объявления.

Парсер написан под обычную верстку (мобильная сильно отличается)

Последние правки 2022-02-16

## Библиотеки и настройки

In [1]:
#Загружаю нужные библиотеки

import pandas as pd
import math

import datetime

# регулярки
import re


# для юзер-агента
from fake_useragent import UserAgent

# прогресс-бар
from tqdm.notebook import trange, tqdm

# для пауз при парсинге
from time import sleep

# селениум
from selenium import webdriver
from selenium.webdriver.chrome.service import Service

from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [2]:
# настройки юзер-агента

ua = UserAgent()

# юзер-агент для Хрома
user_agent = ua.chrome

In [3]:
# тестовая страница для Селениума

test_url = 'https://intoli.com/blog/not-possible-to-block-chrome-headless/chrome-headless-test.html'

In [4]:
# путь к файлам

file_path = 'C:/00_Data/' + input() + '/'
proxy_list_path = 'C:/work/keys/proxy_list.txt' # путь и названия со списком прокси

temp_data


In [5]:
# загружаю список прокси
# список простой текстовый файл, который содержит прокси в формате: адрес:порт логин:пароль

proxy_list = []

with open(proxy_list_path, 'r') as f:
    for line in f:
        proxy_list += [line.split()]
        
print('Рабочих прокси — ' + str(len(proxy_list)))

Рабочих прокси — 9


In [6]:
# номер прокси

print('Номер прокси, от 0 до ' + str(len(proxy_list) - 1))
proxy_number = int(input())

Номер прокси, от 0 до 8
8


In [35]:
# настройки веб-драйвера, чтобы скрыть автоматическое управление

opt = webdriver.ChromeOptions()
#opt.add_argument(f'user-agent={user_agent}')
opt.add_argument('--disable-blink-features')
opt.add_argument('--disable-blink-features=AutomationControlled')
opt.add_argument(f'--proxy-server={proxy_list[proxy_number][0]}') # "девственный" прокси(:
opt.add_experimental_option("excludeSwitches", ["enable-automation"]) # отключает строку про удаленное управление
opt.add_experimental_option('useAutomationExtension', False)

In [36]:
print('Login: ' + proxy_list[proxy_number][1])
print('Pass: ' + proxy_list[proxy_number][2])

Login: tw1QwuaQ
Pass: EJ1SMvdd


In [42]:
# запускаю драйвер, проверяю настройки

#s=Service('C:/Users/bgi/Anaconda3/chromedriver/chromedriver.exe')
#driver = webdriver.Chrome(service=s, options=opt)

driver = webdriver.Chrome(options=opt)
driver.get(test_url)
driver.maximize_window() # раскрываю окно на максимум

## Собираю ссылки

In [10]:
items_find = 'Военная атрибутика'

In [23]:
item_cat = input()

Военное, Другие страны, до 1945


In [15]:
# ID категории

category_id = 14401

Навигация на Мешке — по количеству товаров: к количеству, указанному в конце урла, плюсуется 24.

In [20]:
# начальная страница

start_url = f'https://meshok.net/listing?condition=N&good={category_id}&pN='

In [21]:
driver.get(start_url + '0')
sleep(3)

In [22]:
page_number = int(driver.find_element(By.CLASS_NAME, 'buttons_1qQzv')
                  .text.split('\n')[0]
                  .replace('Страница 1 из ', ''))

print('Количество страниц: ' + str(page_number))

Количество страниц: 494


In [24]:
item_number = 0 # счетчик товаров
data_url = [] # список для данных

for new_page in tqdm(range(page_number)):
    
    driver.get(start_url + str(item_number))
    item_number += 24
    sleep(3)    

    item_blocks = driver.find_elements(By.CLASS_NAME, 'wrapper_18RL-')

    for block in item_blocks:

        name = block.find_element(By.CLASS_NAME, 'itemTitle_2gcl1').text

        ad_url = block.find_element(By.TAG_NAME, 'a[class="itemContent_1rBtL"]').get_attribute('href')

        current_price = block.find_element(By.CLASS_NAME, 'price_3O10j').text

        try:
            blitz_price = block.find_element(By.CLASS_NAME, 'strikePrice_2hGyd').text

        except:
            blitz_price = 'none'

        seller_string = block.find_element(By.CLASS_NAME, 'sellerLink_2rTFC').text.split('\n')
        seller = seller_string[0]
        seller_ads_all = seller_string[1]

        seller_url = block.find_element(By.CLASS_NAME, 'sellerLink_2rTFC').get_attribute('href')

        location = block.find_element(By.CLASS_NAME, 'delivery_185Kn').text.split('\n')

        city = location[0]

        delivery = location[1]

        data_url.append([item_cat, name, current_price, blitz_price,
                         seller, city, delivery, seller_ads_all,
                         ad_url, seller_url])  

  0%|          | 0/494 [00:00<?, ?it/s]

In [25]:
# преобразовываю в датафрейм

head = ['item_cat', 'name', 'current_price', 'blitz_price',
        'seller', 'city', 'delivery', 'seller_ads_all',
        'ad_url', 'seller_url']

work_urls = pd.DataFrame(data_url, columns=head)

In [26]:
work_urls.sample(3)

,item_cat,name,current_price,blitz_price,seller,city,delivery,seller_ads_all,ad_url,seller_url
5705,"Военное, Другие страны, до 1945",Немецкие штаны Смотри и другие мои лоты с 1 рубля,1 700 $,none,АлександрОмск,Омск,Уточняйте у продавца,192,https://meshok.net/item/107310568_%D0%9D%D0%B5...,https://meshok.net/info/733039
9120,"Военное, Другие страны, до 1945","Венгерская каска м37, Венгрия ww2 ВОВ, в краск...",7 500 ₽,none,Leyt-sov,Москва,Самовывоз,508,https://meshok.net/item/171814899_%D0%92%D0%B5...,https://meshok.net/info/354864
5610,"Военное, Другие страны, до 1945",Königsberg ! Пепельница окопная GERMANIA ! С м...,650 ₽,none,wiking Ich,Калининград,300 ₽,643,https://meshok.net/item/251694419_K%C3%B6nigsb...,https://meshok.net/info/491137


In [27]:
work_urls.shape

(11703, 10)

In [28]:
work_urls = work_urls[~work_urls.name.str.contains('книг', case=False)]
work_urls = work_urls[~work_urls.name.str.contains('альбом', case=False)]
work_urls = work_urls[~work_urls.name.str.contains('каталог', case=False)]
work_urls = work_urls[~work_urls.name.str.contains('карта', case=False)]
work_urls = work_urls[~work_urls.name.str.contains('история', case=False)]
work_urls = work_urls[~work_urls.name.str.contains('археология СССР', case=False)]
work_urls = work_urls[~(
    (work_urls.name.str.contains('археолог', case=False)) &
    (work_urls.name.str.contains(' кн', case=False)))]
work_urls = work_urls[~work_urls.name.str.contains('путеводитель', case=False)]
work_urls = work_urls[~work_urls.name.str.contains('сборник', case=False)]
work_urls = work_urls[~work_urls.name.str.contains('издание', case=False)]
work_urls = work_urls[~work_urls.name.str.contains('наука', case=False)]
work_urls = work_urls[~work_urls.name.str.contains(' Кн', case=True)]
work_urls = work_urls[~work_urls.name.str.contains('копия', case=False)]
work_urls = work_urls[~work_urls.name.str.contains('новодел', case=False)]
work_urls = work_urls[~work_urls.name.str.contains('авторская работа', case=False)]

In [29]:
work_urls.shape

(11435, 10)

In [30]:
#work_urls.to_excel(file_path + f'mechok_{items_find}.xlsx', index=False)

In [45]:
2153 + 201

2354

## Собираю данные объявлений

In [31]:
work_urls = pd.read_excel(file_path + 'Meshok\mechok_Военное_Другие_1945.xlsx')

In [32]:
work_urls.shape

(9282, 10)

In [33]:
data_info = [] # список для данных

In [47]:
for ad_url in tqdm(work_urls.ad_url[3464:]):
    
    driver.get(ad_url)
    sleep(3)
        
    try:
        ad_descripton = driver.find_element(By.TAG_NAME, 'div[itemprop="description"]').text
        
    except:
        ad_descripton = 'none'
    
    try:
        ad_tags = (driver.find_element(By.CLASS_NAME, 'standardPadding_1tPoj')
                   .find_element(By.CLASS_NAME, 'container_1bS6g')
                   .text.replace('\n', ';'))
    except:
        ad_tags = 'none'
        
    ad_date = (driver.find_element(By.CLASS_NAME, 'placementDate_jKzv0')
               .find_element(By.TAG_NAME, 'span').get_attribute('title'))
    
    data_info.append([ad_url, ad_descripton, ad_tags, ad_date])

  0%|          | 0/5818 [00:00<?, ?it/s]

In [48]:
# преобразовываю в датафрейм

head = ['ad_url', 'ad_descripton', 'ad_tags', 'ad_date']

work_data = pd.DataFrame(data_info, columns=head)

In [49]:
work_data.shape

(9273, 4)

In [50]:
work_data.drop_duplicates(inplace=True)

In [51]:
work_data.shape

(9264, 4)

In [52]:
work_data.head()

,ad_url,ad_descripton,ad_tags,ad_date
0,https://meshok.net/item/263460313_%D0%9A%D0%9E...,Здравствуйте уважаемые коллекционеры!\n\nВсе в...,,19 февр. 2022 г. 13:47 GMT+03:00
1,https://meshok.net/item/234406241_%D0%9A%D0%B0...,Из семьи потомков лётчика-аса Чернобая Андрея ...,вторая мировая война;Германия;Третий Рейх;люфт...,2 февр. 2022 г. 20:28 GMT+03:00
2,https://meshok.net/item/29876883_%D0%9A%D0%9E%...,"""КОНТЕЙНЕР ДЛЯ ПРОТИВОГАЗА"". Немецкий. Третий ...",контейнер;противогаз;3-й Рейх;Третий Рейх;Герм...,31 янв. 2022 г. 23:31 GMT+03:00
3,https://meshok.net/item/127355417_%D0%9E%D0%BA...,"см.фото\nДелая ставку, Вы соглашаетесь с прави...",третий рейх;окопное творчество;военные вещи;ко...,1 февр. 2022 г. 21:26 GMT+03:00
4,https://meshok.net/item/220526429_%D0%9E%D1%80...,"""Орден Короны Румынии"", рыцарский крест 5 степ...",коллекционирование;оригинал;раритет;рейх;редко...,24 янв. 2022 г. 15:07 GMT+03:00


In [53]:
work_urls.head(1)

,item_cat,name,current_price,blitz_price,seller,city,delivery,seller_ads_all,ad_url,seller_url
0,"Военное, Другие страны, до 1945",- КОМПАС E.R.WATTS & SON. LONDON Тетюхе 1936 г...,4 000 ₽,none,V_I,Уссурийск,500 ₽,3199,https://meshok.net/item/263460313_%D0%9A%D0%9E...,https://meshok.net/info/478262


In [54]:
work = work_urls.merge(work_data, how='left', on='ad_url')

In [57]:
# Записываю в файл

work.to_csv(file_path + 'meshok_' + items_find + '.csv', encoding='utf-8', index=False)

In [56]:
driver.close()